In [1]:
import sys
import requests
from requests import request
import concurrent.futures
import pdb
requests.packages.urllib3.disable_warnings()

In [2]:
fname = '/Users/brownm28/Documents/2019-Jan-31_rna_fusion/bs_id_check/bgi_manifest.csv'
file_list = fname
bs_attrs = ['external_aliquot_id', 'analyte_type', 'source_text_tissue_type', 'source_text_tumor_descriptor', 'composition', 'external_sample_id',]
pt_attrs = ['external_id', 'gender', 'ethnicity', 'race']
dx_attrs = ['source_text_diagnosis', 'source_text_tumor_location', 'age_at_event_days']
gf_attrs = ['platform', 'experiment_strategy']
url = 'https://kf-api-dataservice.kidsfirstdrc.org/'
out_fh = open('/Users/brownm28/Documents/2019-Jan-31_rna_fusion/bs_id_check/bgi_ds_update.txt', 'w')
out_fh.write('Cavatica file name\tBS_ID\tPT_ID\t' + '\t'.join(bs_attrs) + '\t' + '\t'.join(pt_attrs) + '\t' + '\t'.join(dx_attrs) + '\t + ''\t'.join(gf_attrs) + '\n')




281

In [3]:
def check_gf_file(gf_url, gf_name, url):
    try:
        all_gf = request('GET', gf_url, verify=False)
        # pdb.set_trace()
        for entry in all_gf.json()['results']:
            if entry['file_name'] == gf_name:
                # experiment details moved
                sg_info = request('GET', url + entry['_links']['sequencing_experiments'])
                return sg_info.json()['results'][0]
    except Exception as e:
        sys.stderr.write(str(e)+ '\n')
        sys.stderr.write('ERROR: Could not get info from ' + gf_url + ' for ' + gf_name + '\n')
        return {}
    return matched


In [4]:
def query_dataservice_bs_id(url, cur_line, bs_attrs, pt_attrs, dx_attrs, gf_attrs):
    info = cur_line.split(',')
    bs_col = 3
    gf_col = 1
    bs_id = info[bs_col]
    gf_name = info[gf_col]
    bs_id = bs_id.rstrip('\n')
    bs_url = url + '/biospecimens/' + bs_id
    # sys.stderr.write('Processing ' + gf_name + '\n')
    try:
        bs_info = request('GET', bs_url, verify=False)
        result = []
        if bs_info.json()['_status']['code'] == 404:
            result.append(bs_info.json()['_status']['message'])
            sys.stderr.write(bs_id + ' not found!\n')
            return bs_id, result
        gf_url = url + bs_info.json()['_links']['genomic_files']
        gf_surf = check_gf_file(gf_url, gf_name, url)
        f = 0
        if gf_surf:
            f = 1
        else:
            sys.stderr.write('ERROR: bs id for ' + gf_name + ' does NOT match DS entry\n' )
            result = gf_name + '\tERROR\n'
        if f == 1:
            dx_url = url + bs_info.json()['_links']['diagnoses']
            dx_dict = {}
            dx_obj = request('GET', dx_url, verify=False) if len(dx_attrs) > 0 else 'NoDX'
            # dir(bs_info)
            pt_url = bs_info.json()['_links']['participant']
            pt_info = request('GET', url + pt_url, verify=False)
            result.append(pt_info.json()['results']['kf_id'])
            for attr in bs_attrs:
                # sys.stderr.write(attr + ': ')
                res = bs_info.json()['results'][attr]
                if res is None:
                    res = 'NULL'
                # sys.stderr.write(res + '\n')
                result.append(str(res))
            for attr in pt_attrs:
                # sys.stderr.write(attr + ': ')
                res = pt_info.json()['results'][attr]
                if res is None:
                    res = 'NULL'
                # sys.stderr.write(res + '\n')
                result.append(res)
            for attr in dx_attrs:
                dx_dict[attr] = []
                for cur_res in dx_obj.json()['results']:
                    dx_dict[attr].append(str(cur_res[attr]))
                result.append(','.join(dx_dict[attr]))
            for attr in gf_attrs:
                res = gf_surf[attr]
                if type(gf_surf[attr]) is list:
                    res = ','.join(gf_surf[attr]) 
                result.append(res)
    except Exception as e:
        sys.stderr.write(str(e) + '\n')
        sys.stderr.write('Failed while processing ' + gf_url + '\n')
        sys.exit(1)
    return gf_name, bs_id, result
    

In [5]:
in_file = open(fname)
head = next(in_file)
x = 1
m = 50
th = 16
#for line in in_file:
#    query_dataservice_bs_id(url, line, bs_attrs, pt_attrs, dx_attrs, gf_attrs)
with concurrent.futures.ThreadPoolExecutor(th) as bs_exec:
    bs_results = {
    bs_exec.submit(query_dataservice_bs_id, url, line, bs_attrs, pt_attrs, dx_attrs, gf_attrs): line for line in in_file}
    for bs_result in concurrent.futures.as_completed(bs_results):
        if x % m == 0:
            sys.stderr.write('Processed ' + str(x) + ' bs ids\n')
            sys.stderr.flush()
        (gf_name, cur_bs_id, bs_info) = bs_result.result()
        try:
            out_fh.write(gf_name + '\t' + cur_bs_id + '\t' + '\t'.join(bs_info) + '\n')
        except:
            sys.stderr.write('Could not output info for ' + cur_bs_id + '\n')
        x += 1
out_fh.close()
sys.stderr.write('Done!')

Processed 50 bs ids
Processed 100 bs ids
Processed 150 bs ids
Done!